In [1]:
# An implementation of the paper "Embedded Deformation for Shape Manipulation (2007)"
import numpy as np
import igl
import meshplot as mp
import scipy.sparse as sp
from sksparse.cholmod import cholesky
from trimesh.sample import sample_surface_even
import trimesh

In [2]:
class deformation:
    def __init__(self, mesh, n=225, k=4):
        self.v = np.array(mesh.vertices)
        self.nodes = np.array(sample_surface_even(mesh, n)[0])
        self.k = k
        self.nv = self.nodes.shape[0] # #nodes
        self.ne = 0 # #edges
        self.alpha = 1.0
        
        self.dists = igl.all_pairs_distances(v, self.nodes, True)
        self.knn = np.argsort(self.dists)[:,:k+1]
        
        self.A = np.zeros((self.nodes.shape[0], self.nodes.shape[0]))
        self.__set_A()
        
        self.R = np.zeros((self.nv, 9))
        self.T = np.zeros((self.nv, 3))
        for i in range(self.nv):
            self.R[i] = np.array([1., 0., 0., 0., 1., 0., 0., 0., 1.])
        
    def __set_A(self):
        for i in range(self.v.shape[0]):
            n = self.knn[i][:-1]
            for j in range(self.k):
                for l in range(j+1, self.k):
                    self.A[n[j], n[l]] = self.A[n[l], n[j]] = 1
        self.ne = int(np.sum(self.A)//2)
        
    def get_Erot(self):
        res = 0.0
        for i in range(self.nv):
            r = self.R[i].reshape(3, 3)
            c1, c2, c3 = r[:, 0], r[:, 1], r[:, 2]
            rot = (c1.dot(c2))**2 + (c1.dot(c3))**2 + (c2.dot(c3))**2 + (c1.dot(c1)-1)**2 + (c2.dot(c2)-1)**2 + (c3.dot(c3)-1)**2
            res += rot
        return res
    
    def get_Ereg(self):
        res = 0.0
        for i in range(self.nv):
            neighbors = np.where(self.A[i])[0]
            for k in neighbors:
                ri, ti = self.R[i].reshape(3, 3), self.T[i]
                tk = self.T[k]
                d = ri.dot(self.nodes[k] - self.nodes[i]) + self.nodes[i] + ti - self.nodes[k] - tk
                res += self.alpha*(d.dot(d))
        return res
    
    def get_Econ(self, handle_indices, handle_vertices):
        res = 0.0
        for i in range(handle_indices.size):
            q = handle_vertices[i]
            v_ = self.predict(self.v[handle_indices[i]])
            res += (v_-q).dot(v_-q)
        return res
    
    def get_weights(self, v):
        dists = np.array([np.linalg.norm(v - self.nodes[i]) for i in range(self.nv)])
        knn = np.argsort(dists)[:self.k+1]
        n, dmax = knn[:-1], dists[knn[-1]]
        weights = np.zeros(self.k)
        for j in range(self.k):
            weights[j] = (1 - dists[knn[j]]/dmax)**2
        weights /= np.sum(weights)
        return weights, n
        
        
    def predict(self, v):
        weights, n = self.get_weights(v)
        res = np.zeros(3)
        for i in range(self.k):
            res += weights[i]*self.__transform(v, n[i])
        return res
        
    def __transform(self, v, i):
        return self.R[i].reshape(3, 3).dot(v - self.nodes[i]) + self.nodes[i] + self.T[i]

In [3]:
class optimization:
    def __init__(self, dg, handle_indices, handle_vertices, fixed_indices, fixed_vertices):
        self.dg = dg
        self.handle_indices = handle_indices
        self.handle_vertices = handle_vertices
        self.fixed_indices = fixed_indices
        self.fixed_vertices = fixed_vertices
        self.p = handle_indices.size
        self.fixed, self.free = self.__fixed_nodes()
        
        self.nv = self.free.size
        self.mp = {}
        
        self.alpha = 1.0
        self.w_rot = 1.0
        self.w_reg = 10.0
        self.w_con = 100.0
        self.eps = 1e-6
        self.m_iteration = 100
        
        self.x = np.zeros(12*self.nv)
        self.__reset_x()
        self.__set_mp()
        self.fx = np.zeros(6*self.dg.nv + 6*self.dg.ne + 3*self.p)
        self.J = np.zeros((6*self.dg.nv + 6*self.dg.ne + 3*self.p, 12*self.nv))
        
    def __set_mp(self):
        for i in range(self.nv):
            self.mp[self.free[i]] = i
    
    def __reset_x(self):
        for i in range(self.nv):
            self.x[12*i] = self.x[12*i+4] = self.x[12*i+8] = 1.0
            
    def __fixed_nodes(self):
        res = set()
        tot = set(np.arange(self.dg.nv))
        for v in self.fixed_vertices:
            _, n = self.dg.get_weights(v)
            for node in n:
                res.add(node)
        
        return np.sort(list(res)), np.sort(list(tot - res))
    
    def __set_trans(self):
        for i in range(self.nv):
            self.dg.R[self.free[i]] = self.x[12*i :12*i+9]
            self.dg.T[self.free[i]] = self.x[12*i+9 :12*i+12]
         
        
    def F(self):
        self.__set_trans()
        self.get_fx()
        self.get_J()
        return self.w_rot*self.dg.get_Erot() + self.w_reg*self.dg.get_Ereg() + self.w_con*self.dg.get_Econ(self.handle_indices, self.handle_vertices)
    
    def get_fx(self):
        self.fx.fill(0.0)
        
        index = 0
        coef = self.w_rot**0.5
        for i in range(self.dg.nv):
            rot = self.dg.R[i].reshape(3, 3)
            c1, c2, c3 = rot[:, 0], rot[:, 1], rot[:, 2]
            self.fx[index] = c1.dot(c2) * coef
            self.fx[index+1] = c1.dot(c3) * coef
            self.fx[index+2] = c2.dot(c3) * coef
            self.fx[index+3] = (c1.dot(c1) - 1) * coef
            self.fx[index+4] = (c2.dot(c2) - 1) * coef
            self.fx[index+5] = (c3.dot(c3) - 1) * coef
            index += 6
        
        coef = (self.alpha * self.w_reg) ** 0.5
        for i in range(self.dg.nv):
            for j in range(self.dg.nv):
                if self.dg.A[i, j]:
                    ri, ti = self.dg.R[i].reshape(3, 3), self.dg.T[i]
                    tj = self.dg.T[j]
                    d =  ri.dot(self.dg.nodes[j] - self.dg.nodes[i]) + self.dg.nodes[i] + ti - self.dg.nodes[j] - tj
                    self.fx[index: index+3] = coef * d
                    index += 3
        
        coef = self.w_con**0.5
        for i in range(self.p):
            q = self.handle_vertices[i]
            v_ = self.dg.predict(self.dg.v[self.handle_indices[i]])
            d = (v_ - q) * coef
            self.fx[index: index+3] = d
            index += 3
            
            
    def get_J(self):
        self.J.fill(0.0)
        for i in range(self.nv):
            j = self.free[i]
            rot = self.dg.R[j]
            temp = np.array([[rot[1], rot[0], 0     , rot[4], rot[3], 0     , rot[7], rot[6], 0     ],
                             [rot[2], 0     , rot[0], rot[5], 0     , rot[3], rot[8], 0     , rot[6]],
                             [0     , rot[2], rot[1], 0     , rot[5], rot[4], 0     , rot[8], rot[7]],
                             [2*rot[0], 0, 0, 2*rot[3], 0, 0, 2*rot[6], 0, 0],
                             [0, 2*rot[1], 0, 0, 2*rot[4], 0, 0, 2*rot[7], 0],
                             [0, 0, 2*rot[2], 0, 0, 2*rot[5], 0, 0, 2*rot[8]]])
            temp *= self.w_rot**0.5
            self.J[6*j: 6*j+6, 12*i: 12*i+9] = temp
        
        coef = (self.alpha * self.w_reg) ** 0.5
        offset = 6*self.dg.nv
        for i in range(self.dg.nv):
            for j in range(self.dg.nv):
                if self.dg.A[i, j]:
                    diff = coef*(self.dg.nodes[j] - self.dg.nodes[i])
                    if i in self.free:
                        ii = self.mp[i]
                        self.J[offset, 12*ii: 12*ii+3] = diff
                        self.J[offset+1, 12*ii+3: 12*ii+6] = diff
                        self.J[offset+2, 12*ii+6: 12*ii+9] = diff
                        self.J[offset, 12*ii+9] = coef
                        self.J[offset+1, 12*ii+10] = coef
                        self.J[offset+2, 12*ii+11] = coef
                    if j in self.free:
                        jj = self.mp[j]
                        self.J[offset, 12*jj+9] = -coef
                        self.J[offset+1, 12*jj+10] = -coef
                        self.J[offset+2, 12*jj+11] = -coef
                    offset += 3
                    
        coef = self.w_con**0.5
        for l in range(self.p):
            w, n = self.dg.get_weights(self.dg.v[self.handle_indices[l]])
            for j in range(self.dg.k):
                index_ = n[j]
                if index_ in self.fixed:
                    continue
                index = self.mp[index_]
                diff = self.dg.v[self.handle_indices[l]] - self.dg.nodes[index_]
                self.J[offset, 12*index: 12*index+3] = w[j] * coef * diff
                self.J[offset+1, 12*index+3: 12*index+6] = w[j] * coef * diff
                self.J[offset+2, 12*index+6: 12*index+9] = w[j] * coef * diff
                self.J[offset, 12*index+9] = w[j] * coef
                self.J[offset+1, 12*index+10] = w[j] * coef
                self.J[offset+2, 12*index+11] = w[j] * coef
            offset += 3
                
    def solve(self):
        self.__reset_x()
        Fx, Fx_ = 0, self.F()
        iterations = 0
        print('iteration:', iterations, 'Fx=', Fx_)
        while abs(Fx_ - Fx) >= self.eps*(1 + Fx_) and iterations < self.m_iteration:
            Fx = Fx_
            JtJ = self.J.T.dot(self.J)
            factor = cholesky(sp.csc_matrix(JtJ))
            delta = factor(-self.J.T.dot(self.fx))
            self.x += delta
            Fx_ = self.F()
            iterations += 1
            print('iteration:', iterations, 'Fx=', Fx_)
            
    def get_mesh(self):
        mesh = self.dg.v.copy()
        mesh[self.fixed_indices] = self.fixed_vertices
        for i in range(mesh.shape[0]):
            if i in self.fixed_indices:
                continue
            mesh[i] = self.dg.predict(mesh[i])
        return mesh

In [4]:
v, f = igl.read_triangle_mesh('data/woody-lo.off')
v -= v.min(axis=0)
v /= v.max()
mesh = trimesh.Trimesh(v, f)

In [5]:
dg = deformation(mesh)
labels = np.load('data/woody-lo.label.npy').astype(int)
handle_indices = np.where(labels==1)[0]
handle_vertices = v[handle_indices]
for i in range(handle_vertices.shape[0]):
    handle_vertices[i, 2] -= 0.3
fixed_indices = np.where(labels==2)[0]
fixed_vertices = v[fixed_indices]

In [6]:
ll =  optimization(dg, handle_indices, handle_vertices, fixed_indices, fixed_vertices)
ll.solve()
xv = ll.get_mesh()

iteration: 0 Fx= 62.999999999999986
iteration: 1 Fx= 47.88598827292296
iteration: 2 Fx= 1.1311608940645925
iteration: 3 Fx= 0.1929538557744588
iteration: 4 Fx= 0.1886992647429233
iteration: 5 Fx= 0.1886425387122491
iteration: 6 Fx= 0.18864181304086383


In [7]:
mp.plot(xv, f)

/Users/zhongyifan/opt/miniconda3/lib/python3.8/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4090491…